### 라이브러리 로드

In [1]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import csv
from pprint import pprint # Json 출력 인덴트 주어 보기 편하게 출력.
import pandas as pd # 데이터프레임 가공을 위한 라이브러리.
import psycopg2 # 데이터베이스 연동을 위한 라이브러리.
from konlpy.tag import Twitter, Kkma # 한글 형태소 분석 위한 라이브러리.

### DB 불러오기

In [2]:
db = psycopg2.connect(host='192.168.1.222', dbname='omop',user='belab',password='qwer1234',port=5433) # 파이썬 db 연동.

### Table 불러오기

In [3]:
pheno = pd.read_sql("""select tt.* from
(
select 'pheno' as gubun, pheno_id as id, '' as naire_name, '' as pheno_name, '' as qualifier_name, pheno as value from munjin.pheno2 
where category = 'clinical finding' and pheno_id > 130 
union all
select 'munjin' as gubun, munjin_id as id ,naire_name, pheno_name, qualifier_name, value from munjin.munjin2 
where qualifier_name in ('Character', 'Aggravated by', 'Provoked by', 'Risk factor', 'Symptom relieving factors')) as tt
order by value collate "C";""", db)

### Twitter api 사용, 증현상/설명값 품사 분석

In [11]:
twitter = Twitter()
pheno['morpheme_list']=pheno['value'].apply(twitter.pos)

### 명사에 해당하는 형태소만 추출

In [37]:
Noun_list=[]

for i in pheno['morpheme_list'] :
    a = []
    for j in i :
        term = j[0]
        pos = j[-1]
        if pos == 'Noun' :
            a.append(term)
            
    Noun_list.append(a)

pheno['noun_list'] = Noun_list

[[], [], [], [], [], [], [], [], [], ['손가락', '포함', '부위', '어려움'], ['손가락', '부위', '어려움'], ['번', '중', '번', '변', '느낌'], ['번', '중', '번', '손가락', '함'], ['번', '중', '번', '항문', '느낌'], ['번', '중', '번', '힘', '함'], [], ['변'], ['가래'], ['가래', '동반'], ['가래', '동반'], ['가려움'], ['때'], ['가슴', '답답'], ['가슴', '함몰'], ['가슴', '느낌'], ['가슴', '느낌'], ['가슴', '느낌'], ['가슴', '느낌'], ['가족', '요로', '종양', '암', '병력'], ['가족', '중', '증상'], ['가족', '콩팥', '질환'], ['가족', '당뇨'], ['각질'], ['간', '섬유화'], ['공간'], ['감각'], ['감각', '이상'], ['감기', '증상'], ['감기', '증상'], ['갑자기'], ['갑자기', '졸림'], ['갑자기', '통증', '반복'], ['갑자기', '상황'], ['갑자기', '때'], ['갑자기', '증상', '때', '허리', '통증'], ['말', '반복'], ['개', '고양이'], ['개', '고양이'], ['개', '고양이', '접촉'], ['거대', '분만'], ['거동', '제한'], ['짐'], ['거품'], ['거품'], ['거품'], ['거품'], ['거품', '뇨'], ['거품', '변'], ['거품', '물음'], ['해도'], [], ['계단', '등', '일상생활', '때'], ['계단', '등', '일상생활', '때'], ['계단', '오를떄'], ['계단', '오를떄'], ['걷기'], ['걷기'], ['걷기'], ['걷기'], ['악화'], ['걸', '때'], ['걸', '때'], ['걸', '때', '통증', '악화'], ['걸', '수', '정도', '발목', '불', '안정']

### CSV 저장

In [39]:
pheno.to_csv('211014_명사_추출본.csv', encoding='euc-kr')